# Tikhonov regularization, $e^+e^-\rightarrow\eta\pi^+\pi^-$ (1)
In this notebook we obtain the $e^+e^-\rightarrow\eta\pi^+\pi^-$ Born cross section using the ***Tikhonov regularization method***. Visible cross section data is generated using the model Born crosection. Here we consider only a relatively simple model $\rho(770),\,\rho(1450)\rightarrow\eta\rho(770)$. The statistical uncertainties of the generated visible cross section are proportional to the square roots of the cross section values. In this notebook, the following parameters are constant:
- $50$ equidistant c.m. energy points, 
- $\varepsilon(x,s)=1$.

In this Jupiter Notebook, the regularization parameter is set manually.

Regularization leads to a biased numerical solution, so the covariance matrix of the Born cross section is incorrect.

In [ ]:
import numpy as np
import json
import seaborn as sns
from PyISR import ISRSolverTikhonov
from ROOT import TFile
import matplotlib
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.sans-serif'] = ['Tahoma', 'DejaVu Sans',
                                          'Lucida Grande', 'Verdana']

Remove scrolling:

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%matplotlib inline

Loading the model Born cross section

In [ ]:
# Opening the file containing the model cross section
fl = TFile.Open('data/born_cs_etapipi_simple_model.root', 'read')
# Cloning the model cross section function
mBCsFcn = fl.Get('f_bcs').Clone()
# Vectorizing the model Born cross section function
mBCsVFcn = np.vectorize(lambda en: mBCsFcn.Eval(en))
# Closing the file
fl.Close()

Saving the current Matplotlib style:

In [ ]:
# Current Matplotlib style parameters
mplStyleParams = dict(matplotlib.rcParams)
# Deprecated parameters
deprecatedParams = ['animation.avconv_args', 'animation.avconv_path', 
                    'animation.html_args', 'keymap.all_axes',
                   'savefig.jpeg_quality', 'text.latex.preview']
# Remove deprecated parameters in oder to avoid warnings
_ = list(map(lambda key : mplStyleParams.pop(key, None), deprecatedParams))

Function for reading visible cross section data:

In [ ]:
def readVCS(path):
    fl = TFile.Open(path, "read")
    # Getting a pointer to the visible cross section in the form of TGraphErrors
    gvcs = fl.Get('vcs')
    # Number of c.m. energy points
    n = gvcs.GetN()
    # Reading c.m. energy array
    energy = np.frombuffer(gvcs.GetX(), dtype=np.float64, count=n)
    # Reading array of c.m. energy errors
    energyErr = np.frombuffer(gvcs.GetEX(), dtype=np.float64, count=n)
    # Reading visible cross section array
    vcs = np.frombuffer(gvcs.GetY(), dtype=np.float64, count=n)
    # Reading array of visible cross section errors
    vcsErr = np.frombuffer(gvcs.GetEY(), dtype=np.float64, count=n)
    fl.Close()
    return energy, vcs, energyErr, vcsErr

Function for obtaining numerical solution:

In [ ]:
def solve(energy, vcs, energyErr, vcsErr, 
          enabled_energy_spread=False, 
          threshold_energy=0.827,
          reg_param = 0.,
          interp=None,
          efficiency=lambda x, en: 1.0):
    n = energy.shape[0]
    solver = ISRSolverTikhonov(n, energy, vcs, 
                              energyErr, vcsErr, 
                              threshold_energy, efficiency,
                              enabled_energy_spread)
    if type(interp) == str:
        with open(interp, 'r') as jfl:
            settings = json.load(jfl)
        
        print('Interpolation settings:')
        print(settings)
        solver.set_interp_settings(settings)
    elif type(interp) == list:
        print('Interpolation settings:')
        print(interp)
        solver.set_interp_settings(interp)
    
    solver.reg_param = reg_param
    solver.solve()
    return solver

Function for cross section plotting:

In [ ]:
def csPlot(solver, title='Cross sections', fontsize=24):
    # Getting c.m. energy array
    ecm = solver.ecm()
    matplotlib.rcParams.update(mplStyleParams)
    f, (ax0, ax1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]}, sharex=True,
                                figsize=(9, 10))
    f.suptitle(title, fontsize=fontsize)
    ax1.tick_params(axis='both', which='major', labelsize=fontsize)
    ecm_dense = np.linspace(np.min(ecm), np.max(ecm), ecm.shape[0] * 20)
    interp = np.vectorize(lambda en: solver.interp_eval(en))(ecm_dense)
    ax0.errorbar(ecm, solver.vcs(), 
                xerr=solver.ecm_err(),
                yerr=solver.vcs_err(), fmt='o',
                markersize=5, capsize=3,
                label='Visible cross section', zorder=0)
    bcs_err = yerr=np.sqrt(np.diag(solver.bcs_cov_matrix()))
    ax0.errorbar(ecm, solver.bcs(), yerr=bcs_err, fmt='o',
                 markersize=5, capsize=3,
                label='Born cross section', zorder=1)
    ax0.plot(ecm_dense, interp, 'b--', label='Interpolation of the Born cross section', zorder=2)
    ax0.plot(ecm_dense, mBCsVFcn(ecm_dense), 'r-', label='Model Born cross section', zorder=3)
    mBCs_at_ecm =  mBCsVFcn(ecm)
    ax1.errorbar(ecm, solver.bcs() / mBCs_at_ecm, yerr=bcs_err / mBCs_at_ecm, fmt='o',
                 markersize=5, capsize=3,
                 label=r'Ratio $\frac{\sigma_{\rm B}}{\sigma^{\rm model}_{\rm B}}$',
            zorder=0)
    ax1.set_xlabel(r'$\sqrt{s}$ (GeV)', fontsize=fontsize)
    ax0.set_ylabel('cross section (nb)', fontsize=fontsize)
    ax0.legend(fontsize=fontsize, bbox_to_anchor=(1.05, 1))

Function for plotting matrices:

In [ ]:
def matrixPlot(mx, title='', fontsize=24, fontscale=2.0):
    sns.set(font_scale=fontscale)
    f, ax = plt.subplots(figsize=(9, 7))
    f.suptitle(title, fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize=fontsize)
    sns.heatmap(mx, ax=ax, square=True)

## $\sigma_E = 10\text{ MeV}$

### Regularization parameter $\lambda = 0$
In this case, there is no regularization, i.e. in fact, the naive method is used. 

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_10MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True)
print('Condition number: {}'.format(solver.condnum_eval()))
csPlot(solver)
matrixPlot(solver.intop_matrix(), title=r'Matrix of the system of linear equations, $\hat{\mathcal{G}}\hat{\mathcal{F}}$')
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

### Regularization parameter $\lambda=10^{-2}$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_10MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True, reg_param=1.e-2)
csPlot(solver)
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

### Regularization parameter $\lambda=0.1$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_10MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True, reg_param=1.e-1)
csPlot(solver)
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

### Regularization parameter $\lambda=1$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_10MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True, reg_param=1.)
csPlot(solver)
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

### Regularization parameter $\lambda=10$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_10MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True, reg_param=10.)
csPlot(solver)
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

### Regularization parameter $\lambda = 10^{2}$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_10MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True, reg_param=1.e+2)
csPlot(solver)
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

## $\sigma_E = 20\text{ MeV}$

### Regularization parameter $\lambda = 0$
In this case, there is no regularization, i.e. in fact, the naive method is used. 

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_20MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True)
print('Condition number: {}'.format(solver.condnum_eval()))
csPlot(solver)
matrixPlot(solver.intop_matrix(), title=r'Matrix of the system of linear equations, $\hat{\mathcal{G}}\hat{\mathcal{F}}$')
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

### Regularization parameter $\lambda=10^{-3}$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_20MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True, reg_param=1.e-3)
csPlot(solver)
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

### Regularization parameter $\lambda=10^{-2}$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_20MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True, reg_param=1.e-2)
csPlot(solver)
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

### Regularization parameter $\lambda=0.1$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_20MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True, reg_param=1.e-1)
csPlot(solver)
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

### Regularization parameter $\lambda=1$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_20MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True, reg_param=1.)
csPlot(solver)
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

### Regularization parameter $\lambda=10$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_20MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True, reg_param=10.)
csPlot(solver)
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

## $\sigma_E = 5\text{ MeV}$

### Regularization parameter $\lambda = 0$
In this case, there is no regularization, i.e. in fact, the naive method is used. 

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_5MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True)
print('Condition number: {}'.format(solver.condnum_eval()))
csPlot(solver)
matrixPlot(solver.intop_matrix(), title=r'Matrix of the system of linear equations, $\hat{\mathcal{G}}\hat{\mathcal{F}}$')
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')

### Regularization parameter $\lambda = 10^{-1}$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_energy_spread_5MeV.root'
solver = solve(*readVCS(input_path), enabled_energy_spread=True, reg_param=1.e-1)
csPlot(solver)
matrixPlot(solver.bcs_cov_matrix(), title=r'Covariance matrix of the Born cross section')